In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mmh3
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
cov = 10
kmer = 20
sim = 24
sim_path = "../arabidopsis_sim_data/sim_" + str(sim)
pop = 0

In [18]:
info_path = sim_path + "/sim_info_" + str(sim) + ".csv"
info = pd.read_csv(info_path)
ind = info.iloc[0,4]
ind
# info

10

In [19]:
info

,id,pop,pop_size,generations,sampled individuals,mutation rate,coverage,k-length,average pi
0,24,0,100,1000,10,0.00003,10,10,0.0192


In [20]:
kmer_folder = sim_path + "/kmer_counts_x" + str(cov) + "_k" + str(kmer) + "/P" + str(pop) +"/"
i=0
j=2

In [21]:
file1 = kmer_folder + "n" + str(i) + ".txt"
df1 = pd.read_csv(file1,sep="\t",header=None,names=["kmer","counts"])
file2 = kmer_folder + "n" + str(j) + ".txt"
df2 = pd.read_csv(file2,sep="\t",header=None,names=["kmer","counts"])
inter_file = kmer_folder + "n" + str(i) + "_inter_n" + str(j) +  ".txt"
df_inter = pd.read_csv(inter_file,sep="\t",header=None,names=["kmer","counts"])

In [22]:
df1 = df1[df1['counts'] > 4].reset_index().drop("index",axis=1)
df2 = df2[df2['counts'] > 4].reset_index().drop("index",axis=1)
df_inter1 = df_inter[df_inter['counts'] > 4].reset_index().drop("index",axis=1)

In [23]:
def CalcBCTwoIndividuals(df1,df2,df_inter):
    print("Calculating pair...")
    # df1 = pd.read_csv(file1,sep="\t",header=None,names=["kmer","counts"])
    sum1 = sum(df1.counts)

    # df2 = pd.read_csv(file2,sep="\t",header=None,names=["kmer","counts"])
    sum2 = sum(df2.counts)

    # df_inter = pd.read_csv(inter_file,sep="\t",header=None,names=["kmer","counts"])
    sum_inter = sum(df_inter.counts)

    BC = 1 - (2*sum_inter/(sum1 + sum2))

    return BC

In [24]:
BC = CalcBCTwoIndividuals(df1,df2,df_inter)
BC

Calculating pair...


0.20041953193897166

In [25]:
df1.shape[0]

89252

In [26]:
def CalcBFTwoIndividuals(df1,df2,h):
    print("Hashing...")

    num_hash = h
    total = df1.shape[0]
    array_size = 10000

    array1 = np.zeros(array_size,dtype=np.int32)
    array2 = np.zeros(array_size,dtype=np.int32)

    kmers1 = df1.kmer
    counts1 = df1.counts
    total = df1.shape[0]

    for i in range(0,total):
        for k in range(0, num_hash):
            # print("working on i: %d, k: %d" % (i,k))
            index = mmh3.hash(kmers1[i],k,signed=False)%array_size
            array1[index] += counts1[i]

    kmers2 = df2.kmer
    counts2 = df2.counts
    total = df2.shape[0]

    for i in range(0,total):
        for k in range(0, num_hash):
            index = mmh3.hash(kmers2[i],k,signed=False)%array_size
            array2[index] += counts2[i]

    # print("Calculating..")

    cosine = cosine_similarity([array1],[array2])

    return cosine[0][0]

In [27]:
BF = CalcBFTwoIndividuals(df1,df2,4)
1-BF

Hashing...


0.006061397520889189

In [28]:
# df1.set_index('kmer',inplace=True)
# df2.set_index('kmer',inplace=True)
df1

,kmer,counts
0,AAAAAAAAAAAAAAAAAAAG,9
1,AAAAAAAAAAAAAAAAAAGA,6
2,AAAAAAAAAAAAAAAAAAGG,10
3,AAAAAAAAAAAAAAAAAGAG,6
4,AAAAAAAAAAAAAAAAAGGC,10
...,...,...
89247,TTTTTACTTTTAACTAAAAA,8
89248,TTTTTATGTAAAAAAAAAAA,12
89249,TTTTTGTTTACAAAAAAAAA,11
89250,TTTTTTATGTAAAAAAAAAA,12


In [29]:
# df2.set_index('kmer',inplace=True)
res = pd.concat([df1,df2],axis=1)
res = res.fillna(0)

In [30]:
array1 = res.iloc[:,0].values
array2 = res.iloc[:,1].values
CS = cosine_similarity([array1],[array2])
CS = CS[0][0]

ValueError: could not convert string to float: 'AAAAAAAAAAAAAAAAAAAG'

In [ ]:
1-CS

0.20038031108988674